# Fetch Something Something Data-set

[Something Something v2 webpage](https://developer.qualcomm.com/software/ai-datasets/something-something)

The video data is provided as one large TGZ archive, split into parts of 1 GB maximum. The total download size is 19.4 GB. The archive contains webm-files, using the VP9 codec, with a height of 240px. Files are numbered from 1 to 220847.

For each video in the training and validation sets there is an object annotation in addition to the video label, if applicable. For example, for a label like "Putting [something] onto [something]," there is also an annotated version, such as "Putting a cup onto a table." In total, there are 318,572 annotations involving 30,408 unique objects.


[download instructions](./links/20bn-something-something_download_instructions_-_091622.pdf)

In [ ]:
import os
import subprocess
import shutil

import pandas as pd

In [ ]:
os.mkdir("./data/")
os.mkdir("./data/extracted")

In [ ]:
zip_files = [file for file in os.listdir('./') if '.zip' in file]

In [ ]:
for file in zip_files:
    subprocess.call(
        f'unzip {file} -d ./data/extracted',
        shell=True
    )

In [ ]:
os.chdir("./data/")
print(os.listdir())
subprocess.call(
    f'cat ./extracted/20bn-something-something-v2-?? | tar zx',
    shell=True
)
print(f"number of files extracted: {len(os.listdir('20bn-something-something-v2'))}")
os.chdir("../")

## Convert To Avi Format

In [3]:
INPUT_FORMAT = 'webm'
OUTPUT_FORMAT = 'avi'

target_dir = "./data/" + OUTPUT_FORMAT + "/"
print(os.listdir())
cat_dir = "./data/20bn-something-something-v2"

failed = []
for f in os.listdir(cat_dir):
    f2 = f.replace("." + INPUT_FORMAT, "." + OUTPUT_FORMAT)
    try: 
        if not os.path.exists(target_dir + f2):
            subprocess.call(
                f'ffmpeg -i {cat_dir}/{f} {target_dir}/{f2}',
                shell=True
            )
        else:
            print(f"{f2} already exists")
    except:
        failed.append(f"{f}")
    
    print(f"failed: {str(failed)}")

['20bn-something-something-v2-16.zip', '20bn-something-something-v2-17.zip', 'run_ffmpeg.py', 'data', 'data_right_left_up_down_1200', '20bn-something-something-v2-05.zip', '20bn-something-something-v2-11.zip', '20bn-something-something-v2-06.zip', '20bn-something-something-v2-18.zip', '20bn-something-something-v2-08.zip', '20bn-something-something-v2-02.zip', '20bn-something-something-v2-10.zip', '20bn-something-something-v2-00.zip', '__pycache__', '20bn-something-something-v2-09.zip', '2_Simple_Spatial_Compression.ipynb', '20bn-something-something-v2-03.zip', 'requirements.txt', '20bn-something-something-v2-01.zip', '12-Point Ethics Checklist for UG and MSc Projects-Jan2022_EDITED.docx', '.gitignore', 'README.md', '20bn-something-something-v2-13.zip', '20bn-something-something-v2-04.zip', '20bn-something-something-v2-12.zip', 'Checklist for Full University Ethics Sub-Committee Review_SIGNED.pdf', 'gcsaccesskey.json', '0_setup.ipynb', '20bn-something-something-v2-19.zip', '20bn-somethi

In [ ]:
assert len(os.listdir("./data/20bn-something-something-v2/")) == 220847

In [ ]:
os.listdir("./data/20bn-something-something-v2/")

## Extract target file classes

In [ ]:
file_prefix = "right_left_up_down_1200"
data_folder = "data_" + file_prefix
data_source = "dat"

if "data_" + file_prefix not in os.listdir():
    os.mkdir(data_folder)
    os.mkdir(data_folder + "/train")
    os.mkdir(data_folder + "/test")
    os.mkdir(data_folder + "/validation")
    shutil.copy("indx_df.csv", data_folder)

indx_df = pd.read_csv(f"{data_folder}/indx_df.csv")

indx_df

In [ ]:
train_file_ids = list(indx_df[indx_df["type"] == "train"]["name"])
test_file_ids = list(indx_df[indx_df["type"] == "test"]["name"])
validation_file_ids = list(indx_df[indx_df["type"] == "validation"]["name"])

def relocate_files_to_ttv_folder(data_source, folder_prefix, type, file_ids):
    current_files = os.listdir(f"{folder_prefix}/{type}/")

    print(current_files)

    for f_id in file_ids:
        if f_id not in current_files:
            shutil.copy(f"{folder_prefix}/{type}/{f_id}", data_folder)


relocate_files_to_ttv_folder(data_folder, "train", train_file_ids)
relocate_files_to_ttv_folder(data_folder, "test", test_file_ids)
relocate_files_to_ttv_folder(data_folder, "validation", validation_file_ids)